In [1]:
import time
import random
import math

In [2]:
people = [('Seymour', 'BOS'),
          ('Franny', 'DAL'),
          ('Zooey', 'CAK'),
          ('Walt', 'MIA'),
          ('Buddy', 'ORD'),
          ('Les', 'OMA')]

destination = 'LGA'

In [3]:
flights = {}

for line in open('schedule.txt'):
    origin, dest, depart, arrive, price = line.strip().split(',')
    flights.setdefault((origin, dest), [])
    # 将航班详情添加到航班列表中
    flights[(origin, dest)].append((depart, arrive, int(price)))

# 获取分钟数
def getminutes(t):
    x = time.strptime(t, '%H:%M')
    return x[3]*60 +x[4]

# 打印时间表
def printschedule(r):
    for d in range(int(len(r)/2)):
        name = people[d][0]
        origin = people[d][1]
        out = flights[(origin, destination)][r[2*d]]
        ret = flights[(destination, origin)][r[2*d+1]]
        print('%10s%10s %5s-%5s $%3s %5s-%5s $%3s' % (name, origin, out[0], out[1], out[2], ret[0], ret[1], ret[2]))

In [4]:
# 成本函数
def schedulecost(sol):
    totalprice = 0
    latestarrival = 0
    earliestdep = 24*60
    
    for d in range(int(len(sol)/2)):
        # 得到往返航班
        origin = people[d][1]
        outbound = flights[(origin, destination)][int(sol[2*d])]
        returnf = flights[(destination, origin)][int(sol[2*d+1])]
        
        # 总价格等于所有往返航班价格之和
        totalprice += outbound[2]
        totalprice += returnf[2]
        
        # 记录最晚到达时间和最早离开时间
        if latestarrival < getminutes(outbound[1]):
            latestarrival = getminutes(outbound[1])
        if earliestdep > getminutes(returnf[0]):
            earliestdep = getminutes(returnf[0])
    
    # 每个人必须在机场等待，直到最后一个人到达
    # 每个人必须在相同时间到达机场，等候往返航班
    # 在机场的等待时间价值1美元
    totalwait = 0
    for d in range(int(len(sol)/2)):
        origin = people[d][1]
        outbound = flights[(origin, destination)][int(sol[2*d])]
        returnf = flights[(destination, origin)][int(sol[2*d+1])]
        totalwait += latestarrival - getminutes(outbound[1])
        totalwait += getminutes(returnf[0]) - earliestdep
        
    if latestarrival < earliestdep:
        totalprice += 50
    
    return totalprice+totalwait

In [5]:
s = [1,4,3,2,7,3,6,3,2,4,5,3]
print(schedulecost(s))
printschedule(s)

4585
   Seymour       BOS  8:04-10:11 $ 95 12:08-14:05 $142
    Franny       DAL 10:30-14:57 $290  9:49-13:51 $229
     Zooey       CAK 17:08-19:08 $262 10:32-13:16 $139
      Walt       MIA 15:34-18:11 $326 11:08-14:38 $262
     Buddy       ORD  9:42-11:32 $169 12:08-14:47 $231
       Les       OMA 13:37-15:08 $250 11:07-13:24 $171


In [6]:
# 随机搜索
def randomoptimize(domain, costf):
    best = 99999999
    bestr = None
    for i in range(1000):
        # 创建一个随机解
        r = [random.randint(domain[i][0], domain[i][1]) for i in range(len(domain))]
        # 得到成本
        cost = costf(r)
        
        if cost < best:
            best = cost
            bestr = r
    return r

In [7]:
domain = [(0,9)] * (len(people) * 2)
ss = randomoptimize(domain, schedulecost)
print(schedulecost(ss))
printschedule(ss)

5940
   Seymour       BOS  6:17- 8:26 $ 89 13:39-15:30 $ 74
    Franny       DAL 12:19-15:25 $342 17:14-20:59 $277
     Zooey       CAK 12:08-14:59 $149 18:17-21:04 $259
      Walt       MIA 17:07-20:04 $291 15:23-18:49 $150
     Buddy       ORD  8:25-10:34 $157 10:33-13:11 $132
       Les       OMA 18:12-20:17 $242  9:31-11:43 $210


In [8]:
# 爬山法
# 基于时间成本，寻找更合适的相邻安排
def hillclimb(domain, costf):
    # 创建一个随机解
    sol = [random.randint(domain[i][0], domain[i][1]) for i in range(len(domain))]
    
    while True:
        # 创建相邻解的列表
        neighbors = []
        for j in range(len(domain)):
            # 在每个方向上偏离原值一点
            if sol[j] > domain[j][0]:
                neighbors.append(sol[0:j] + [sol[j]-1] + sol[j+1:])
            
            if sol[j] < domain[j][1]:
                neighbors.append(sol[0:j] + [sol[j]+1] + sol[j+1:])
                
        current = costf(sol)
        best = current
        for j in range(len(neighbors)):
            cost = costf(neighbors[j])
            if cost < best:
                best = cost
                sol = neighbors[j]
        
        if best == current:
            break
            
    return sol

In [9]:
ss = hillclimb(domain, schedulecost)
print(schedulecost(ss))
printschedule(ss)

3181
   Seymour       BOS 12:34-15:02 $109 10:33-12:03 $ 74
    Franny       DAL 10:30-14:57 $290 14:20-17:32 $332
     Zooey       CAK  8:27-10:45 $139 10:32-13:16 $139
      Walt       MIA 11:28-14:40 $248 15:23-18:49 $150
     Buddy       ORD  9:42-11:32 $169 10:33-13:11 $132
       Les       OMA 12:18-14:56 $172 11:07-13:24 $171
